In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import sys
sys.path.append('../src_variable')

import network as nw 
import road as rd 
import junction as jn 
import traffic_lights as tl 
import cProfile
import pstats

In [1]:
n = 10
m = 7
L = 50
N = 10
Vmax = torch.tensor([30.])
control_points = []
distribution = [[0.25, 0.25, 0.25, 0.25],
                [0.25, 0.25, 0.25, 0.25],
                [0.25, 0.25, 0.25, 0.25],
                [0.25, 0.25, 0.25, 0.25]]

fwd_hor = [[None for _ in range(m+1)] for _ in range(n)]
fwd_ver = [[None for _ in range(m)] for _ in range(n+1)]
bwd_hor = [[None for _ in range(m+1)] for _ in range(n)]
bwd_ver = [[None for _ in range(m)] for _ in range(n+1)]

# Creating the junctions
junctions = [[None for _ in range(m)] for _ in range(n)]

for i in range(n): 
    for j in range(m):
        # Node at position (i,j)
        
        if i == 0:
            # The first row of nodes
            if j == 0:
                # The first corner of grid
                # Need to create all 8 roads
                
                # To the left
                fwd_hor[0][0] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(-1, 0), right_pos=(0, 0),
                                        inflow = 0.2)
                bwd_hor[0][0] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, 0), right_pos=(-1, 0),
                                        inflow = 0.2)

                # Above
                fwd_ver[0][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, -1), right_pos=(0, 0),
                                        inflow = 0.2)
                bwd_ver[0][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, 0), right_pos=(0, -1),
                                        inflow = 0.2)

                # To the right
                fwd_hor[0][1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, 0), right_pos=(1, 0),
                                        inflow = 0.2)
                bwd_hor[0][1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(1, 0), right_pos=(0, 0),
                                        inflow = 0.2)
            
                # Below
                fwd_ver[1][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, 0), right_pos=(0, 1),
                                        inflow = 0.2)
                bwd_ver[1][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, 1), right_pos=(0, 0),
                                        inflow = 0.2)

                # Add to junction 
                junctions[0][0] = jn.Junction([fwd_hor[0][0], fwd_ver[0][0], bwd_hor[0][1], bwd_ver[1][0],
                                               fwd_hor[0][1], fwd_ver[1][0], bwd_hor[0][0], bwd_ver[0][0]],
                                               [0,1,2,3], [4,5,6,7], distribution, [], [])

            else:
                # The rest of the first row
                # Roads to the left are already created
                # position = (0, j) 

                # To the left [0][j], already created

                # Above
                fwd_ver[0][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, -1), right_pos=(j, 0),
                                        inflow = 0.2)
                bwd_ver[0][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, 0), right_pos=(j, -1),
                                        inflow = 0.2)

                # To the right
                fwd_hor[0][j+1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, 0), right_pos=(j+1, 0),
                                        inflow = 0.2)
                bwd_hor[0][j+1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j+1, 0), right_pos=(j, 0),
                                        inflow = 0.2)

                # Below
                fwd_ver[1][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, 0), right_pos=(j, 1),
                                        inflow = 0.2)
                bwd_ver[1][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, 1), right_pos=(j, 0),
                                        inflow = 0.2)

                # Add to junction
                junctions[0][j] = jn.Junction([fwd_hor[0][j], fwd_ver[0][j], bwd_hor[0][j+1], bwd_ver[1][j],
                                               fwd_hor[0][j+1], fwd_ver[1][j], bwd_hor[0][j], bwd_ver[0][j]],
                                               [0,1,2,3], [4,5,6,7], distribution, [], [])

        elif j == 0:
            # The first column of nodes (except corner)
            # Roads above are already created
            # position = (i, 0)

            # To the left
            fwd_hor[i][0] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(-1, i), right_pos=(0, i),
                                        inflow = 0.2)
            bwd_hor[i][0] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, i), right_pos=(-1, i),
                                        inflow = 0.2)

            # Above [i][0], already created

            # To the right
            fwd_hor[i][1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, i), right_pos=(1, i),
                                        inflow = 0.2)
            bwd_hor[i][1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(1, i), right_pos=(0, i),
                                        inflow = 0.2)

            # Below
            fwd_ver[i+1][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, i), right_pos=(0, i+1),
                                        inflow = 0.2)
            bwd_ver[i+1][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, i+1), right_pos=(0, i),
                                        inflow = 0.2)

            # Add to junction
            junctions[i][0] = jn.Junction([fwd_hor[i][0], fwd_ver[i][0], bwd_hor[i][1], bwd_ver[i+1][0],
                                           fwd_hor[i][1], fwd_ver[i+1][0], bwd_hor[i][0], bwd_ver[i][0]],
                                           [0,1,2,3], [4,5,6,7], distribution, [], [])
        
        else:
            # node is not at left or upper edge of grid
            # position = (i, j)

            # To the left, already created

            # Above, already created

            # To the right
            fwd_hor[i][j+1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, i), right_pos=(j+1, i),
                                        inflow = 0.2)
            bwd_hor[i][j+1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j+1, i), right_pos=(j, i),
                                        inflow = 0.2)

            # Below
            fwd_ver[i+1][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, i), right_pos=(j, i+1),
                                        inflow = 0.2)
            bwd_ver[i+1][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, i+1), right_pos=(j, i),
                                        inflow = 0.2)

            # Add to junction
            junctions[i][j] = jn.Junction([fwd_hor[i][j], fwd_ver[i][j], bwd_hor[i][j+1], bwd_ver[i+1][j],
                                           fwd_hor[i][j+1], fwd_ver[i+1][j], bwd_hor[i][j], bwd_ver[i][j]],
                                           [0,1,2,3], [4,5,6,7], distribution, [], [])

NameError: name 'torch' is not defined

In [ ]:
roads = [road for row in fwd_hor for road in row] + [road for row in fwd_ver for road in row] +\
        [road for row in bwd_hor for road in row] + [road for row in bwd_ver for road in row]

jncs = [junction for row in junctions for junction in row]
# network = nw.Network([road for road in fwd_hor])